
## Basis Swap Par Spreads

## Bootstrapping technique to derive discount curve

The first step of our project consists in extracting a discount curve (which is characterized by pillar dates and the corresponding discount factors) from OIS market data, via a process called bootstrapping. Bootstrapping describes a procedure used to calculate the zero-coupon yield curve from market figures.

In [2]:
from datetime import date
from finmarkets import DiscountCurve,OvernightIndexSwap,generate_swap_dates
from curve_data import euribor_3m_curve, euribor_6m_curve,euribor_1m_curve, discount_curve
from ois_data import basis_swaps, observation_date, quotes

ModuleNotFoundError: No module named 'finmarkets'

## Description of the classes that we used


Before starting with our code, we imported some fundamental classes from the finmarkets file. We started with the DiscountCurve class. The main attribute of this class are:

* a list pillar dates specifying the value dates of the given discount factors t0,....,tn.
* a list of given discount factors D(t0), ..., D(tn).
* a pricing date (' observation_date') which corresponds to t0.

Since the discount factor can be expressed as $ D = \mathrm{exp}( -r ( T-t)) $ , we wrote a function called df for calculating a discount factor at any date, using log-linear interpolation.

Moreover, the formula to calculate the forward rates can be found exploiting the no arbitrage condition $ (1 + r1T1) (1 + r1,2 (T2 - T1)) = 1 + r2T2 $

Another important class used is the OvernightIndexSwap. OIS are products which pay a floating coupon, determined by overnight rate fixings over the reference periods, against a fixed coupon. By definition an OIS is defined by:

* a notional amount N
* a start date d0
* a sequence of payment dates d1, ..., dn
* a fixed rate K

Our goal will be to take a series of OIS quotations ('ois_data.quotes'), and determine the OIS's NPV given a certain discount curve. However, using the Bootstrapping technique we are going to somehow reverse this procedure. Therefore, the final aim will be to get the discount curve such that it prices correctly each OIS ( by minimizing the sum of the square NPVs).

Finally, we imported generateswap dates, a function which given a start date and the number of months, returns a list of dates of annual frequency starting from the start date and ending after the specified number of months.

In [ ]:
d = {}
pillar_dates = [observation_date]
swaps = [] 
for quote in quotes:
    swap = OvernightIndexSwap(
    1e6,
    generate_swap_dates(
    observation_date,
    quote['maturity']
    ),
    0.01 * quote['rate']
    )
    swaps.append(swap)
    pillar_dates.append(swap.payment_dates[-1])

pillar_dates = sorted(pillar_dates)
n_df_vector = len(pillar_dates)

type(pillar_dates), len(pillar_dates), pillar_dates[0], pillar_dates[-1]

def objective_function(x):
    curve = DiscountCurve(
        observation_date,
        pillar_dates,
        x
)
    sum_sq = 0.0
    for swap in swaps:
        sum_sq += swap.npv(curve) ** 2
    return sum_sq

from scipy.optimize import minimize
from matplotlib import pyplot as plt

x0 = [1.0 for i in range(n_df_vector)]

bounds = [(0.01, 100.0) for i in range(n_df_vector)]
bounds[0] = (1.0, 1.0)
result = minimize(objective_function, x0, bounds=bounds)

print(result.x)
discount_curve = DiscountCurve(observation_date, pillar_dates, result.x)
print(pillar_dates)

### What the minimization algorithm actually does:

* It starts with our initial guess of the parameters and compute the objective function value;
* Move the parameters to reduce the value of the objective function;
* Repeat above step until further changes of the parameters do not bring additional reduction of the objective function.

We can use a graph to check whether the minimization algorithm actually worked or not.

In [1]:
from matplotlib import pyplot as plt

plt.plot(result.x,)

NameError: name 'result' is not defined

## Par Spreads and NPV

A Tenor Basis Swap, also known as a floating-floating interest rate swap, is a financial instrument whereby floating cashflows from two different interest rates are exchanged, typically floating Libor indices of the same currency are exchanged e.g. 3M Libor vs 6M Libor cash- flows.

The contract parameters are:

* start date d0
* notional N
    floating rate long tenor (months)
    floating rate short tenor (months)
    maturity (years)

The floating legs pay the reference LIBOR fixing at a frequency equal to the tenor of the floating rate.

The basis spread informs investors what adjustment or spread to add to the lower tenor Libor index so that the interest payments on both legs of the basis swap are equivalent.

We can easiy calculate S as: par%20spread%203%20x6.png

In the general case we write par%20spread%20general.png

Where long index denotes the Libor index with the longer tenor and short index the Libor index with the shorter tenor.
